<a href="https://colab.research.google.com/github/vrnTEK/Advance-Deep-Learning/blob/master/Assignment_1_Part_A_Multi_Instance_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment-1 Part-A

## A) Multi Instance Learning:

### Definition and Understanding:

Consider this you have an album with many images you have clicked and you want to determine how many of those images have cats. The Question is how to determine this.

One approach could be to create a CNN model and train it on multiple cats images and pass your images to identify which images have cats. This approach is good provided the image size is not too large and fits within the memory.

But if the image exceeds the memory size then what can we do. Perhaps increase more memory (This is inefficient as there will always be scenarios with huge data size which is much bigger than the memory). So how solve this problem.

One thing we can do is that we can split multiple images into smaller chunks and determine if any one of them has a cat in it. If even one small piece has a cat (or cat like structure) the image will be classified as positive (image with a cat) else it will be negative.

This approach of weakly identifying the labels is called as Multi Instance Learning where a single instance is divided into multiple instance and then classification will be carried on them.